In [1]:
!pip install -q transformers

In [2]:
import torch
import time
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import copy
import datetime
import shutil
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, TensorDataset, RandomSampler, SequentialSampler
from torch.cuda.amp import autocast, GradScaler
from torch import nn
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup, BertForSequenceClassification, BertConfig
from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
# Check that we are using 100% of GPU memory footprint support libraries/code
# from https://github.com/patrickvonplaten/notebooks/blob/master/PyTorch_Reformer.ipynb
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip -q install gputil
!pip -q install psutil
!pip -q install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 15.7 GB  | Proc size: 687.3 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


In [4]:
df = pd.read_csv('../input/full-in-house-dataset/Final_In_house_dataset.csv')
df.sample(10)

,Context,Acronym,Target,Definition,Label
58376,2 Terni 04/24/19 4^3 06:39/00:18 3.955 kg F Va...,PE,Pulmonary embolism,Pulmonary embolism is a blockage in one of the...,False
37943,General/bilateral: • MS: • MS: Right lower ext...,MS,Musculoskeletal system,Your musculoskeletal system includes your bone...,True
91817,150.1- 428.1 Left VF I,VF,Ventricular fibrillation,Ventricular fibrillation is a type of abnormal...,False
40558,Intractable PE (CMS-hcc) [G40.119],PE,Pulmonary emphysema,Pulmonary emphysema is a chronic lung conditio...,False
11755,Pt continued to have CP which required her to ...,CP,Chest pain,"Discomfort in the chest including a dull ache,...",True
81934,"Awaiting imaging, unclear etiology of symptoms...",SCT,Spinocerebellar tract,The spinocerebellar tracts are afferent neuron...,False
68227,PVD (PVD) (CMS/HCC) I73.9,PVD,Primary affective disorders,The main types of affective disorders are depr...,False
53730,• PE - Annual physical exam e Retinopathy due ...,PE,Pleural effusion,"Pleural effusion, sometimes referred to as “wa...",False
46565,"1C] overweight(CLAMI, breast cancerCL1c) s/p b...",PE,Physical examination,"The routine physical, also known as general me...",True
97372,He was discharged with Foley catheter in place...,VT,Ventricular tachycardia,A condition in which the lower chambers of the...,False


In [5]:
df_train, df_test = train_test_split(df, test_size = 0.3, stratify = df['Label'], random_state = 11)

In [6]:
df_train.reset_index(inplace = True)
df_test.reset_index(inplace = True)

In [7]:
class CustomDataset(Dataset):

    def __init__(self, data, maxlen, with_labels=True, bert_model='microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'):

        self.data = data  # pandas dataframe
        self.tokenizer = AutoTokenizer.from_pretrained(bert_model, do_lower_case=True)  
        self.sent1 = self.data['Context'].values
        self.sent2 = self.data['Definition'].values
        self.maxlen = maxlen
        self.with_labels = with_labels 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        # Selecting sentence1 and sentence2 at the specified index in the data frame
        sent1 = str(self.sent1[index])
        sent2 = str(self.sent2[index])

        # Tokenize the pair of sentences to get token ids, attention masks and token type ids
        encoded_pair = self.tokenizer(sent1, sent2, 
                                      padding='max_length',  # Pad to max_length
                                      truncation=True,  # Truncate to max_length
                                      max_length=self.maxlen,  
                                      return_tensors='pt')  # Return torch.Tensor objects
        
        token_ids = encoded_pair['input_ids'].squeeze(0)  # tensor of token ids
        attn_masks = encoded_pair['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
        #token_type_ids = encoded_pair['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

        if self.with_labels:  # True if the dataset has labels
            label = self.data.loc[index, 'Label']
            return token_ids, attn_masks,  int(label)
        else:
            return token_ids, attn_masks
    

In [8]:
def get_data_loaders(batch_size, train_data, validation_data):
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size, num_workers=5)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size, num_workers=5)
    return train_dataloader, validation_dataloader

In [9]:
def set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    

def evaluate(net, device, criterion, dataloader):
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    net.eval()
    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    valid_loss = 0.0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Evaluate data for one epoch
    for batch in tqdm(dataloader):
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        
            outputs = net(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]
        loss = criterion(logits, b_labels) #Criterion is crossentropy loss
        valid_loss += loss.item()
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
      
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        # Track the number of batches
        nb_eval_steps += 1
    # Report the final accuracy for this validation run.
    avg_valid_loss = valid_loss / len(dataloader) 
    avg_eval_accuracy = eval_accuracy/nb_eval_steps
    return avg_valid_loss, avg_eval_accuracy

In [10]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds = elapsed_rounded))


def save_ckp(state, is_best, checkpoint_path, best_model_path):
    f_path = checkpoint_path
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_path
        shutil.copyfile(f_path, best_fpath)


def train(model, epochs, train_dataloader, validation_dataloader, optimizer, scheduler, device, criterion, 
                                                            checkpoint_path, best_model_path):
    loss_values = []
    valid_loss_min = np.inf
    for epoch_i in range(0, epochs):
        
        # ========================================
        #               Training
        # ========================================
        
        # Perform one full pass over the training set.
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')
        # Measure how long the training epoch takes.
        t0 = time.time()
        # Reset the total loss for this epoch.
        total_loss = 0.0
        valid_loss = 0.0
        # Put the model into training mode.
        model.train()
        # For each batch of training data...
        for step, batch in enumerate(tqdm(train_dataloader)):
            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)
                
                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            # Unpack this training batch from our dataloader. 
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            model.zero_grad()        
            # Perform a forward pass (evaluate the model on this training batch).
            # This will return the loss (rather than the model output) because we have provided the labels.
            outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
            

            loss = outputs[0]
            total_loss += loss.item()
            # Perform a backward pass to calculate the gradients.
            loss.backward()
            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            # Update parameters and take a step using the computed gradient.
            optimizer.step()
            # Update the learning rate.
            scheduler.step()
        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)            
        
        # Store the loss value for plotting the learning curve.
        loss_values.append(avg_train_loss)
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
            
        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        avg_valid_loss, avg_eval_accuracy = evaluate(model, device, criterion, validation_dataloader)
        print("  Accuracy: {0:.2f}".format(avg_eval_accuracy))
        print("  Average validation loss: {0:.2f}".format(avg_valid_loss))
        print("  Validation took: {:}".format(format_time(time.time() - t0)))
        checkpoint = {
                'epoch': epoch_i + 1,
                'valid_loss_min': avg_valid_loss,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
            }
            
        save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        if avg_valid_loss <= valid_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, avg_valid_loss))
                save_ckp(checkpoint, True, checkpoint_path, best_model_path)
                valid_loss_min = avg_valid_loss
        
        
    print("")
    print("Training complete!")
    return model

In [11]:
model = BertForSequenceClassification.from_pretrained(
  'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext',
    num_labels = 2,  
    output_attentions = False, 
    output_hidden_states = False,
)

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    
model.to(device)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Ber

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [12]:
train_data = CustomDataset(df_train, 512, model)
validation_data = CustomDataset(df_test, 512, model)
train_dataloader, validation_dataloader = get_data_loaders(16, train_data, validation_data)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/221k [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [13]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

params = list(model.named_parameters())
for name, param in params[-5:]:
    param.requires_grad = True
for name, param in params[:-6]:
    param.requires_grad = True

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

epochs = 3
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
criterion = nn.CrossEntropyLoss()

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [14]:
#  Set all seeds to make reproducible results
set_seed(1)
%mkdir checkpoint best_model
checkpoint_path = "./checkpoint/current_checkpoint.pt"
best_model_path = "./best_model/best_model_exp_3_pubmedbert_inhouse.pt"
model = train(model, epochs, train_dataloader, validation_dataloader, optimizer, scheduler, device, criterion, 
                                                            checkpoint_path, best_model_path)


======== Epoch 1 / 3 ========
Training...


  1%|          | 40/4392 [00:35<1:02:09,  1.17it/s]

  Batch    40  of  4,392.    Elapsed: 0:00:36.


  2%|▏         | 80/4392 [01:10<1:01:37,  1.17it/s]

  Batch    80  of  4,392.    Elapsed: 0:01:10.


...

  Batch 4,360  of  4,392.    Elapsed: 1:02:56.


100%|██████████| 4392/4392 [1:03:23<00:00,  1.15it/s]



  Average training loss: 0.05
  Training epcoh took: 1:03:23

Running Validation...


100%|██████████| 1883/1883 [09:07<00:00,  3.44it/s]


  Accuracy: 0.98
  Average validation loss: 0.11
  Validation took: 1:12:31

Training complete!


In [15]:
# Set the batch size.  
batch_size = 16
# Create the DataLoader.
validation_data = CustomDataset(df_test, 512, model)
prediction_sampler = SequentialSampler(validation_data)
prediction_dataloader = DataLoader(validation_data, sampler=prediction_sampler, batch_size=batch_size)

In [16]:
prediction_inputs = validation_data
# =validation_masks
# =validation_labels
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions , true_labels = [], []
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
print('DONE.')

Predicting labels for 30,114 test sentences...
DONE.


In [17]:
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]
# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)
print('MCC: %.3f' % mcc)

MCC: 0.939


In [18]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
y_pred = flat_predictions
y_true = flat_true_labels
# precision_recall_fscore_support(y_true, y_pred, average='macro')
precision_recall_fscore_support(y_true, y_pred, average='macro')
# precision_recall_fscore_support(y_true, y_pred, average='weighted')

(0.9729475268689662, 0.9656450022171612, 0.9692162074352064, None)

In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98     22041
           1       0.97      0.94      0.95      8073

    accuracy                           0.98     30114
   macro avg       0.97      0.97      0.97     30114
weighted avg       0.98      0.98      0.98     30114

